In [4]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 

In [2]:
import torch

from source.nn.model import NormalizedEightPointNet
from source.datasets.colmap_dataset import ColmapDataset
from torch.utils.data import DataLoader

In [3]:
dataset = ColmapDataset("/Users/apple/Downloads/Family", 20, 1000)
loader = DataLoader(dataset, batch_size=1)

In [4]:
model = NormalizedEightPointNet(3)

model.load_state_dict(torch.load("/Users/apple/Downloads/tanksandtemples_model.pt", map_location='cpu'))
model.to('cpu')
model = model.eval()

In [5]:
from source.utils.eval_utils import compute_error, transform_F_into_image_space

In [7]:
import numpy as np

for kp1, kp2, F, additional_info in loader:
    F_estimates, norm_transform1, norm_transform2, weights = model(kp1, kp2, additional_info)
    
    F_image = transform_F_into_image_space(norm_transform1, norm_transform2, F_estimates[-1])[0].detach()
    
    print(compute_error(kp1[0], kp2[0], F_image, F[0]))
    break

(0.8193430656934306, 0.7059748427672956)


In [46]:
import numpy as np
from source.utils.colmap_utils import quaternion_to_rotation_matrix

path = "/Users/apple/Downloads/south-building/sparse"
with open(os.path.join(path, 'cameras.txt'), 'r') as f:
    raw = f.readlines()[3:]
    
camera_intrinsics = {}
for camera in raw:
    camera = camera.split(' ')
    camera_intrinsics[int(camera[0])] = [float(elem) for elem in camera[2:]]

with open(os.path.join(path, 'images.txt'), 'r') as f:
    raw = f.readlines()[4:]

raw_pose = {}
camera = {}
for image in raw[:: 2]:
    image = image.split(' ')
    
    raw_pose[int(image[0])] = [float(elem) for elem in image[1: -2]]
    camera[int(image[0])] = int(image[-2])

intrinsics = {}
poses = {}

for key in raw_pose.keys():
    image_intrinsics = camera_intrinsics[camera[key]]
    K = np.zeros([3, 3])
    K[0, 0] = image_intrinsics[2]
    K[0, 2] = image_intrinsics[4]
    K[1, 1] = image_intrinsics[3]
    K[1, 2] = image_intrinsics[5]
    K[2, 2] = 1
    intrinsics[key] = K

    image_pose = raw_pose[key]
    qvec = image_pose[: 4]
    qvec = qvec / np.linalg.norm(qvec)
    w, x, y, z = qvec
    R = np.array([
        [
            1 - 2 * y * y - 2 * z * z,
            2 * x * y - 2 * z * w,
            2 * x * z + 2 * y * w
        ],
        [
            2 * x * y + 2 * z * w,
            1 - 2 * x * x - 2 * z * z,
            2 * y * z - 2 * x * w
        ],
        [
            2 * x * z - 2 * y * w,
            2 * y * z + 2 * x * w,
            1 - 2 * x * x - 2 * y * y
        ]
    ])
    
    t = image_pose[4: 7]
    
    # World-to-Camera pose
    current_pose = np.zeros([4, 4])
    current_pose[: 3, : 3] = R
    current_pose[: 3, 3] = t
    current_pose[3, 3] = 1
    
    poses[key] = current_pose
    


In [47]:
from source.utils.colmap_utils import pair_id_to_image_ids, compose_fundamental_matrix, compute_residual
import sqlite3

connection = sqlite3.connect("/Users/apple/Downloads/south-building/database.db")
cursor = connection.cursor()

cursor.execute("SELECT pair_id, data FROM matches WHERE rows>=?;", (20,))

dataset = []

for row in cursor:
    img1_id, img2_id = pair_id_to_image_ids(row[0])
    
    print(img1_id, img2_id)
    
    K1, K2 = intrinsics[img1_id], intrinsics[img2_id]
    T1, T2 = poses[img1_id], poses[img2_id]
    
    print(K1, K2)
    print(T1, T2)

    F = compose_fundamental_matrix(K1, T1, K2, T2)

    matches = np.fromstring(row[1], dtype=np.uint32).reshape(-1, 2)

    inner_cursor = connection.cursor()
    inner_cursor.execute("SELECT data, cols FROM keypoints WHERE image_id=?;", (img1_id,))

    inner_row = next(inner_cursor)
    kp1 = np.fromstring(inner_row[0], dtype=np.float32).reshape(-1, inner_row[1])

    inner_cursor.execute("SELECT data, cols FROM keypoints WHERE image_id=?;", (img2_id,))

    inner_row = next(inner_cursor)
    kp2 = np.fromstring(inner_row[0], dtype=np.float32).reshape(-1, inner_row[1])

    inner_cursor.execute("SELECT data FROM descriptors WHERE image_id=?;", (img1_id,))

    inner_row = next(inner_cursor)
    descriptor1 = np.float32(np.fromstring(inner_row[0], dtype=np.uint8).reshape(-1, 128))

    inner_cursor.execute("SELECT data FROM descriptors WHERE image_id=?;", (img2_id,))

    inner_row = next(inner_cursor)
    descriptor2 = np.float32(np.fromstring(inner_row[0], dtype=np.uint8).reshape(-1, 128))

    kp1 = kp1[matches[:, 0]]
    kp2 = kp2[matches[:, 1]]

    angle1 = kp1[:, 3]
    angle2 = kp2[:, 3]

    descriptor1 = descriptor1[matches[:, 0]]
    descriptor2 = descriptor2[matches[:, 1]]

    desc_dist = np.sqrt(np.mean((descriptor1 - descriptor2) ** 2, 1))[..., None]
    rel_scale = np.abs(kp1[:, 2] - kp2[:, 2])[..., None]
    rel_orient = np.minimum(np.abs(angle1 - angle2), np.abs(angle2 - angle1))[..., None]

    additional_info = np.hstack((desc_dist, rel_scale, rel_orient))

    kp1 = kp1[:, :2]
    kp2 = kp2[:, :2]

    res = compute_residual(kp1, kp2, F.T)
    residual_mask = res < 1
    
    print(residual_mask)

    if np.sum(residual_mask) >= 20:
        dataset.append([kp1, kp2, F.T, additional_info, residual_mask])
        
    break

cursor.close()
connection.close()

1 8
[[ 2.55968e+03  0.00000e+00  1.15200e+03]
 [ 0.00000e+00  1.53600e+03 -2.04997e-02]
 [ 0.00000e+00  0.00000e+00  1.00000e+00]] [[ 2.55968e+03  0.00000e+00  1.15200e+03]
 [ 0.00000e+00  1.53600e+03 -2.04997e-02]
 [ 0.00000e+00  0.00000e+00  1.00000e+00]]
[[ 0.496396    0.29921895  0.81489817 -0.756852  ]
 [-0.23583271  0.94989697 -0.20513086  0.980926  ]
 [-0.83544835 -0.09035351  0.54209068  3.58659   ]
 [ 0.          0.          0.          1.        ]] [[-9.98922202e-01  6.60832087e-04  4.64111778e-02 -7.80245000e-01]
 [-2.91197365e-02  7.69723476e-01 -6.37712954e-01  1.15655000e+00]
 [-3.61451942e-02 -6.38377110e-01 -7.68874626e-01  3.15961000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[False False False False False False False False False False False False
 False False False False False False False False]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/Library/Frameworks/Python.framework/Versions/3.

In [38]:
len(dataset)

18